## Select the parameters for the experiment and log in the experiment folder

In [1]:
# %%writefile param_sets.json
seed_value = 123  # seed for the experiment
Trial = 3  # number of the experiment

period = 6  # the period to sample the data at. 1 period= 5 minutes

inputcols = ['oat', 'orh', 'sat-oat', 'wbt']  # input predictors
x_loc = [0, 1, 2, 3]  # vars we want to plot in detailed plot
outputcols = ['hwe']  # output targets
input_timesteps = 1  # number of timesteps for the input sequence
output_timesteps = 1  # number of timesteps for the output sequence

# Smoothing
smooth_data = True  # whether to smooth the data or not
order = 5  # order of the filer
T = 300  # sampling_period in seconds
fs = 1 / 300  # sample rate, Hz
cutoff = 0.0001  # desired cutoff frequency of the filter, Hz

# adjust out of phase data
adjust_lag = True  # whether to adjust the lag for certain columns
lag_columns = outputcols  # choose columns to adjust lag
data_lag = -1  # lag by how many periods: negative means shift column upwards

# aggregate data based on period
aggregate_data = True  # aggregate data or not
rolling_sum_target = ['hwe']  # create sum aggregate for these columns
rolling_mean_target = [
    'oat', 'orh', 'sat-oat', 'wbt',
]  # create mean aggregate for these columns

# create temporal batches of data: df2dflist
days, hours = 7, 0

# Values below which we consider as 0 output in hwe units is kBTU in period*5 mins
threshold = 0.5

# Custom way to create Training Data
startweek = 0  # start week; indicates how large training set is
data_weeks = 39  # Create a large initial block startweek-data_weeks weeks of training and testing data
end_week = -1
create_lag = 0  # Create further lags in the output
scaling = True  # Scale the input and output features
feature_range = (0, 1)  # Scaling range
reshaping = True  # reshape data according to (batch_size, time_steps, features)

# model configuration
modelconfig = {
    'lstm_hidden_units': 4,
    'lstm_no_layers': 0,
    'dense_hidden_units': 16,
    'dense_no_layers': 6,
    'retrain_from_layers': 3,
    'train_stateful': False,
    'test_stateful':False,
    'train_batchsize':32,
    'test_batchsize':1, # we are doing online prediction
    'train_epochs': 5000,
}

# wheter doing adaptive for fixed learning
adaptive_control = True  # whether we relearn or keep it fixed
path = '../results/' + outputcols[0] + '_model_{}/'.format(
    Trial) + 'adaptive/' * adaptive_control + 'fixed/' * (1 - adaptive_control)
#!rm -rf ../results/lstm_hwe_trial8/adaptive

#model design considerations
modeldesigndone = False  # whether model will be reinitialized
initial_epoch = 0  # the start epoch number for the training

# These are automatically superseded and ignored if adaptive_control is set to False
retain_prev_model = True  # retain weights of model from previous training
freeze_model = True  # freeze weights of certain layers
reinitialize = True  # reinitialize the weights of certain layers

model_saved = False  # whether model has been saved once
test_model_created = False  # create an idectical model for online predicton

# data used for learning the model
datapath = '../data/processed/buildingdata.pkl'

# additional info
addl = {
    'metainfo': 'create a diff of sat and oat for hot water energy prediction as it is useful. See 1.0.8',
    'names_abreviation': {
        'oat':'Outside Air Temperature',
        'orh':'Outside Air Relative Humidity',
        'sat-oat' : 'Difference of Supply Air and Outside Air Temps ',
        'ghi': 'Global Solar Irradiance',
        'hw_sf':'Hot Water System Flow Rate',
        'hx_vlv1':'Hot Water Valve %',
        'hw_st':'Hot Water Supply Temperature',
        'hwe': 'Hot Water Energy Consumption',
        'wbt': 'Wet Builb Temperature',
    }
}
x_lab = [addl['names_abreviation'][inputcols[i]] for i in x_loc]

## Set Seed in numpy, Keras and TF for reproducability; Import modules; Set GPU configurations

In [2]:
import shutil
import glob
import os
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# Enable '0' or disable '-1' GPU use
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# including the project directory to the notebook level
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import json
from tqdm import tqdm
import parse
import warnings
from matplotlib.dates import date2num

In [3]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    #tf.random.set_seed(seed_value)
    # for later versions: 
    tf.compat.v1.set_random_seed(seed_value)
    config = tf.ConfigProto(log_device_placement=False)
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
    from keras import backend as K
    from nn_source import models as mp

from dataprocess import dataprocessor as dp
from dataprocess import plotutils as pu

Using TensorFlow backend.


In [4]:
# %load_ext tensorboard
# %tensorboard --logdir '../results/lstm_hwe_trial1/loginfo/' --port 8200

## Create Folder to save models and tensorboard logs

In [5]:
# create the results directory
try:
    os.makedirs(path)
except FileExistsError:
    files = os.listdir(path)
    for f in files:
        try:
            shutil.rmtree(path + f)
        except NotADirectoryError:
            os.remove(path + f)
        
os.mkdir(path + 'loginfo')
os.mkdir(path + 'normalplots')
os.mkdir(path + 'detailedplots')

## Save the experiment parameters and configurations

In [6]:
#save the values
paramsdict = {
    
    'seed_value' : seed_value,
    
    'period':period,
    
    'inputcols':inputcols,
    'x_loc': x_loc,
    'outputcols':outputcols,
    'input_timesteps':input_timesteps,
    'output_timesteps':output_timesteps,
    
    'smooth_data': smooth_data,
    'order' : 5,
    'T' : T,
    'fs' : fs,
    'cutoff' : cutoff,
    
    'adjust_lag' : adjust_lag,
    'lag_columns' : lag_columns,
    'data_lag' : data_lag,
    
    'aggregate_data' : aggregate_data,
    'rolling_sum_target' : rolling_sum_target,
    'rolling_mean_target' : rolling_mean_target,
    
    'days':days,
    'hours':hours,
    
    'threshold': threshold,
    
    'startweek': startweek,
    'data_weeks' : data_weeks,
    'end_week':end_week,
    'create_lag' : create_lag,
    'scaling' : scaling,
    'feature_range' : feature_range,
    'reshaping' : reshaping,
    
    'modelconfig' : modelconfig,
    
    'adaptive_control':adaptive_control,
    'path':path,
    
    'modeldesigndone' : modeldesigndone,
    'initial_epoch' : initial_epoch,
    
    'retain_prev_model' : retain_prev_model,
    'freeze_model' : freeze_model,
    'reinitialize' : reinitialize,
    
    'model_saved' : model_saved,
    'test_model_created': test_model_created,
    
    'datapath' : datapath,
    
    'addl' : addl,
}
    
# with open(path+'params.json', 'r') as fp:
#     param2dict = json.load(fp)

with open(path+'params.json', 'w') as fp:
    json.dump(paramsdict, fp, indent=4)

## Read the cleaned data

In [7]:
# read the pickled file for ahu data
df1data = dp.readfile(datapath)

# return pickled df
df1 = df1data.return_df(processmethods=['file2df'])

In [8]:
# read the pickled file for wet bulb data
df2data = dp.readfile('../data/processed/interpolated/wetbulbtemp.pkl')

# return pickled df
df2 = df2data.return_df(processmethods=['file2df'])

In [17]:
df = dp.merge_df_columns([df1,df2])

## Create additional Data columns as needed

In [18]:
df['sat-oat']= df['sat']-df['oat']

## Smooth the data

In [19]:
if smooth_data:
    df = dp.dfsmoothing(df=df,
                        column_names=list(df.columns),
                        order=order,
                        Wn=cutoff,
                        T=T)

## Remove <0 values for output energy

In [20]:
df['hwe'][df['hwe']<=0]=0.001

## Adjust lag for certain columns if needed

In [21]:
if adjust_lag:
    df = dp.createlag(df, lag_columns, lag=data_lag)

## Create aggregate data: aggregate specified columns at specified intervals

In [22]:
# return a new column which is the sum of previous window_size values
def window_sum(df_, window_size: int, column_names: list):
    return df_[column_names].rolling(window=window_size, min_periods=window_size).sum()

# return a new column which is the average of previous window_size values
def window_mean(df_, window_size: int, column_names: list):
    return df_[column_names].rolling(window=window_size, min_periods=window_size).mean()

# rolling_sum_output = ['{}min_{}_sum'.format(5*period,target) for target in rolling_sum_target]
# rolling_mean_output = ['{}min_{}_mean'.format(5*period,target) for target in rolling_mean_target]

if aggregate_data:
    
    # rolling sum
    if rolling_sum_target:
        df[rolling_sum_target] =  window_sum(df, window_size=period, column_names=rolling_sum_target)
    
    # rolling mean
    if rolling_mean_target:
        df[rolling_mean_target] =  window_mean(df, window_size=period, column_names=rolling_mean_target)
    
    df = dp.dropNaNrows(df)
    
    # Sample the data at period intervals
    df = dp.sample_timeseries_df(df, period=period)

In [23]:
# show data
df.describe()

,oat,sat,sat_stpt,orh,hwe,cwe,ghi,avg_stpt,flow,hw_rt,hw_sf,hw_st,hw_s_stp,hx_vlv1,wbt,sat-oat
count,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000,21971.000000
mean,63.626614,70.279652,71.527172,60.052722,20.669878,22.974858,172.949942,71.543912,37.365195,91.813681,8.736043,101.209883,101.228952,25.113443,55.037905,6.652519
std,17.450034,5.522785,2.912957,17.310180,24.971515,17.184686,256.256255,1.049008,30.230293,13.201565,9.362448,14.165990,14.217190,19.756852,14.707197,17.865938
min,17.821972,51.414037,58.605072,17.309428,0.000000,-3.155953,-52.457182,66.099812,-1.813372,64.903137,-2.463692,80.075723,87.778581,-6.725714,14.846887,-25.245728
25%,49.121291,67.350621,72.986150,46.642814,0.353918,6.788249,-0.008301,70.895253,9.579592,82.487393,0.153497,90.170177,90.000000,8.603475,42.724135,-6.342551
50%,65.803442,71.940837,73.000000,59.389471,12.052000,21.382697,14.551819,71.342909,32.586087,87.652988,5.768496,93.361023,91.824746,24.093105,57.820886,2.493411
75%,77.806630,73.671614,73.000000,74.189350,29.194800,38.433394,284.159439,72.485714,63.573852,98.307198,13.121573,112.266250,112.209060,42.577149,68.115798,23.150384
max,97.319737,90.015817,74.044062,95.007589,160.411511,66.524093,973.479051,73.557921,125.398265,135.552460,32.572670,147.479417,148.331329,73.261725,85.891934,55.312001


## Get mean of the entire scaled data

In [ ]:
# get the mean of the outputs for the entire data
dfscaled = ((df-df.min())/(df.max()-df.min()))
threshold_scaled = ((threshold-df.min()['hwe'])/(df.max()['hwe']-df.min()['hwe']))
dfscaled = dfscaled[dfscaled['hwe']>=threshold_scaled]
dfmean = dfscaled.mean() 
mean_output = list(dfmean[outputcols])
mean_input = list(dfmean[inputcols])

## Create temporal chunks of data

In [ ]:
# Creating a list of "days" day dataframes for training
dflist = dp.df2dflist_alt(df[inputcols+outputcols],
                      subsequence=True,
                      period=period,
                      days=days,
                      hours=hours)
print('Length of dflist: {}'.format(len(dflist)))

## Custom way to create Training Data

In [ ]:
assert (input_timesteps == 1) & (
    output_timesteps == 1), "Input and Output timesteps must be 1 for this notebook"

In [ ]:
from pandas import concat
import swifter


def quickmerge(listdf):
    return concat(listdf)


def df2operating_regions(df, column_names, thresholds):
    """
    Select from data frame the operating regions based on threshold
    """
    
    org_shape = df.shape[0]
    
    # select cells to be retained
    constraints = df.swifter.apply(
        lambda row: all([(cell > thresholds) for cell in row[column_names]]),
        axis=1)
    # Drop values set to be rejected
    df = df.drop(df.index[~constraints], axis = 0)
    
    print("Retaining {}% of the data".format(100*df.shape[0]/org_shape))
    
    return df

weeklist = []  # create list of training, testing arrays

In [ ]:
# select and merge data_weeks-1 worth of data
datablock_train_pre = dflist[startweek:data_weeks-1]
datablock_train = quickmerge(datablock_train_pre)
#Select from data frame the operating regions
datablock_train = df2operating_regions(datablock_train, outputcols, threshold)
"""
#divide into contiguous regions
datablock_train = dp.subsequencing(datablock_train, period=period)
#drop smaller blocks
datablock_train = [item for item in datablock_train if item.shape[0] >= 0.05*modelconfig['train_batchsize']]
#merge them together
minibatch_train = quickmerge(datablock_train)
"""
minibatch_train = datablock_train

# select weeks=1 worth of data
datablock_test_pre = dflist[data_weeks-1]
#Select from data frame the operating regions
datablock_test = df2operating_regions(datablock_test_pre, outputcols, threshold)
"""
#divide into contiguous operating region
datablock_test = dp.subsequencing(datablock_test_pre, period=period)
#drop smaller blocks
datablock_test = [item for item in datablock_test if item.shape[0] >= 0.05*modelconfig['test_batchsize']]
#merge them together
minibatch_test = quickmerge(datablock_test)
"""
minibatch_test = datablock_test


# splitvalue
splitvalue = minibatch_test.shape[0]
#merge test and train together
data_block = quickmerge([minibatch_train, minibatch_test])

# create numpy arrays
X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
        data_block,
        predictorcols=inputcols,
        outputcols=outputcols,
        scaling=scaling,
        feature_range=feature_range,
        reshaping=reshaping,
        lag=create_lag,
        split=splitvalue,
    input_timesteps=input_timesteps,
    output_timesteps = output_timesteps
    )

# select test ids for later plots
test_idx = minibatch_test.index

# year and week
yearno = minibatch_test.index[int(splitvalue/2)].year
weekno = minibatch_test.index[int(splitvalue/2)].week

# append them
weeklist.append({
        'Id':'Year-{}-Week-{}'.format(str(yearno), 
                                      str(weekno)),
        'X_train':X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'y_scaler':y_scaler,
        'X_scaler':X_scaler,
        'test_idx':test_idx,
    })


In [ ]:
for weekdata in dflist[data_weeks:end_week]:
    
    # select and merge data_weeks-1 worth of data
    datablock_train_pre = datablock_train_pre[1:]+[datablock_test_pre]
    datablock_train = quickmerge(datablock_train_pre)
    #Select from data frame the operating regions
    datablock_train = df2operating_regions(datablock_train, outputcols, threshold)
    minibatch_train = datablock_train
    
    # select weeks=1 worth of data
    datablock_test_pre = weekdata
    #Select from data frame the operating regions
    datablock_test = df2operating_regions(datablock_test_pre, outputcols, threshold)
    minibatch_test = datablock_test
    
    # splitvalue
    splitvalue = minibatch_test.shape[0]
    #merge test and train together
    data_block = quickmerge([minibatch_train, minibatch_test])

    # and add new week data from weekdata
    X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
        data_block,
        predictorcols=inputcols,
        outputcols=outputcols,
        scaling=scaling,
        feature_range=feature_range,
        reshaping=reshaping,
        lag=create_lag,
        split=splitvalue,
    input_timesteps=input_timesteps,
    output_timesteps = output_timesteps
    )

    # select test ids for later plots
    test_idx = minibatch_test.index
    
    # year and week no
    weekno += 1
    weekno = weekno if weekno%53 != 0 else 1
    yearno = yearno if weekno!= 1 else yearno+1

    weeklist.append({
        'Id':'Year-{}-Week-{}'.format(str(yearno), 
                                      str(weekno)),
        'X_train':X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'y_scaler':y_scaler,
        'X_scaler':X_scaler,
        'test_idx':test_idx,
    })

print('Length of weeklist: {}'.format(len(weeklist)))

## Print size and shape of data to feed to the model for sanity checks

In [ ]:
for week in weeklist:
    for key,value in week.items():
        if (key != 'y_scaler') & (key != 'X_scaler') :
            print("name: {}, shape: {}".format(key, value.shape if not isinstance(value,str) else value))

## Add weekly train test data to modelconfig dictionary for ease of training

In [ ]:
modelconfig['weeklist'] = weeklist

##  Clear the Tensorflow graph from previous training

In [ ]:
K.clear_session()

## Inspect model architecture

In [ ]:
# weekdata = modelconfig['weeklist'][1]
# X_train = weekdata['X_train']
# y_train = weekdata['y_train']
# X_test = weekdata['X_test']
# y_test = weekdata['y_test']
# y_scaler = weekdata['y_scaler']

# #Instantiate learner model
# nn_model = mp.simple_LSTM_model(path,
#                       inputdim=X_train.shape[-1],
#                       outputdim=y_train.shape[-1],
#                       period=period)

# # Desing model architecture
# nn_model.design_network(lstmhiddenlayers=[modelconfig['lstm_hidden_units']] * (2*modelconfig['lstm_no_layers']),
#                    densehiddenlayers=[modelconfig['dense_hidden_units']] * modelconfig['dense_no_layers'],
#                    dropoutlist=[[], []],
#                    batchnormalizelist=[[], []])

# # compile model
# nn_model.model_compile()
# print(K.get_session())

# # nn_model.model.summary()0x7f8d2b89f048  0x7f8ba4767550>

# #Instantiate learner model
# nn_model2 = mp.simple_LSTM_model(path,
#                       inputdim=X_train.shape[-1],
#                       outputdim=y_train.shape[-1],
#                       period=period)

# # Desing model architecture
# nn_model2.design_network(lstmhiddenlayers=[modelconfig['lstm_hidden_units']] * (2*modelconfig['lstm_no_layers']),
#                    densehiddenlayers=[modelconfig['dense_hidden_units']] * modelconfig['dense_no_layers'],
#                    dropoutlist=[[], []],
#                    batchnormalizelist=[[], []])

# # compile model
# nn_model.model_compile()

# print(K.get_session())

## Train the model

In [ ]:
for weekno, BatchData in enumerate(tqdm(modelconfig['weeklist'])):
    
    X_train = BatchData['X_train']
    y_train = BatchData['y_train']
    y_scaler = BatchData['y_scaler']
    X_scaler = BatchData['X_scaler']
    X_test = BatchData['X_test']
    y_test = BatchData['y_test']

    """Begin weekly training"""   
    K.clear_session()
    
    try:
        del nn_model
    except NameError:
        pass

    #Instantiate learner model
    nn_model = mp.regression_nn(path,
                                  inputdim=X_train.shape[-1],
                                  outputdim=y_train.shape[-1],
                                  input_timesteps=input_timesteps,
                                  output_timesteps = output_timesteps,
                                  period=period,
                                  stateful = modelconfig['train_stateful'],
                                  batch_size=modelconfig['train_batchsize'])

    # Desing model architecture
    nn_model.design_network(lstmhiddenlayers=[modelconfig['lstm_hidden_units']] * modelconfig['lstm_no_layers'],
                            densehiddenlayers=[modelconfig['dense_hidden_units']] * modelconfig['dense_no_layers'],
                            dropoutlist=[[], []],
                            batchnormalizelist=[[], []])

    # load the trained model weights if we want to: here some layer weights may be reinitialized; see below
    if model_saved & retain_prev_model:
        nn_model.model.load_weights('IntermediateModel.h5')


    # compile model
    nn_model.model_compile()   

    # train the model for adaptive model and fixed after first round for fixed control
    if adaptive_control | (weekno==0):
        history = nn_model.train_model(X_train,
                                       y_train,
                                       X_test,
                                       y_test,
                                       epochs=modelconfig['train_epochs'],
                                       initial_epoch=initial_epoch)
        try:
            initial_epoch += len(history.history['loss'])
        except KeyError:
            pass

        # save the model only if trained at least once- needed for prediction model
        nn_model.model.save('IntermediateModel.h5')
        model_saved = True     
    """End Weekly Training"""
    
    
    """Begin Last Week Prediction"""
    try:
        del nn_model_pred
    except NameError:
        pass

    # Separate predictor for predicting online: only difference is test batch size
    nn_model_pred = mp.regression_nn(path,
                                       inputdim=X_test.shape[-1],
                                       outputdim=y_test.shape[-1],
                                       input_timesteps=input_timesteps,
                                       output_timesteps = output_timesteps,
                                       period=period,
                                       stateful = modelconfig['test_stateful'],
                                       batch_size=modelconfig['test_batchsize'])

    # Desing model architecture
    nn_model_pred.design_network(lstmhiddenlayers=[modelconfig['lstm_hidden_units']] * modelconfig['lstm_no_layers'],
                            densehiddenlayers=[modelconfig['dense_hidden_units']] * modelconfig['dense_no_layers'],
                            dropoutlist=[[], []],
                            batchnormalizelist=[[], []])


    # load the trained model weights
    nn_model_pred.model.load_weights('IntermediateModel.h5')
    # compile model
    nn_model_pred.model_compile()

    # evaluate the model for metrics at this stage
    # train and test plots as well as logged errors inside the text file
    preds_test = nn_model_pred.evaluate_model( X_test,
                                               y_test,
                                               y_scaler,
                                               save_plot_loc=path+'normalplots/',
                                               scaling=True,
                                               saveplot=True,
                                               Idx=BatchData['Id'],
                                               outputdim_names=[addl['names_abreviation'][outputcols[0]]],
                                               output_mean = mean_output)

    # do a detailed plot instead
    pu.regression_plot(period * 5,
                    xs = date2num(list(BatchData['test_idx'])),
                    outputdim=len(outputcols),
                    output_timesteps=output_timesteps,
                    input_timesteps=input_timesteps,
                    pred=preds_test,
                    target=y_test,
                    X_var=X_test,
                    x_loc=x_loc,
                    x_lab=x_lab,
                    saveloc=path + 'detailedplots/',
                    scaling=True,
                    Xscaler=X_scaler,
                    yscaler=y_scaler,
                    outputdim_names=[addl['names_abreviation'][outputcols[0]]],
                    typeofplot='test',
                    Idx=BatchData['Id'])
    """End Last Week Prediction"""

    """Only execute when we are freezing LSTM and just training on Dense"""
    if adaptive_control:
        
        # freeze all but dense layers at the top and compile with new weights
        if freeze_model:
            for layer in nn_model.model.layers[:modelconfig['retrain_from_layers']]:
                layer.trainable = False

        # for relearning, reinitialize top few layers
        if reinitialize:
            for layer in nn_model.model.layers[modelconfig['retrain_from_layers']:]:
                layer.kernel.initializer.run(session=K.get_session())
                layer.bias.initializer.run(session=K.get_session())

        # recompile model
        if freeze_model | reinitialize:
            nn_model.model_compile()
            
            # save the model- needed for Keras limitations: Tensorboard crashes if we relearn on original model with
            # reinitialized weights; solution: we create new model and load these weights
            nn_model.model.save('IntermediateModel.h5')
            model_saved = True
            
            freeze_model = False
            reinitialize = False

## Plot the CVRMSE error on chunks of temporal data

In [ ]:
metricname = 'cvrmse'
# Open a file
fo = open(path + "{}min Results_File.txt".format(5 * period), "r")
print("Name of the file: ", fo.name)
lines = fo.readlines()

parse_format = 'Year-{}-Week-{}-Time Step {}: {} RMSE={} |{} CVRMSE={} |{} MAE={}'
stats = {
    'Train': {
        'rmse': [],
        'cvrmse': [],
        'mae': []
    },
    'Test': {
        'rmse': [],
        'cvrmse': [],
        'mae': []
    }
}
xticklist = []
counter = 0
for line in lines:
    p = parse.parse(parse_format, line)
    stats[p[3]]['rmse'].append(float(p[4]))
    stats[p[5]]['cvrmse'].append(float(p[6]))
    stats[p[7]]['mae'].append(float(p[8]))
    if counter % 1 == 0:
        xticklist.append('Year-{}-Week-{}'.format(p[0], p[1]))
    counter += 1
fo.close()
metric_list = stats['Test'][metricname]

# max_metric = max(cvrmse_list)
# metric_list = [i if i <= 30 else (10*(i-30)/(max_cvrmse-30))+30 for i in cvrmse_list]

cvrmse = sum(metric_list) / len(metric_list)
# from dataprocess import plotutils as pu
plot_args = dict(
    bars=metric_list,
    color='goldenrod',
    bar_label=metricname,
    saveloc=path,
    smoothcurve=True,
    bar_annotate=True,
    saveplot=True,
    xlabel='Week of year',
    ylabel='cvrmse error in percentage',
    title=
    'Weekly CVRMSE Error for Hot Water Energy Model at {5:}min(s) intervals \n [{0:} \
    layers of {1:}-unit lstm, {2:} layers of {3:}-unit dense] \n Average CVRMSE Error {4:.2f}%'
    .format(modelconfig['lstm_no_layers'], modelconfig['lstm_hidden_units'],
            modelconfig['dense_no_layers'], modelconfig['dense_hidden_units'],
            cvrmse, 5 * period),
    xticklist=xticklist,
    plotwidth=17,
    plotheight=6,
    fontsize=16,
    savetitle=metricname+' plot.png')
pu.regression_bar_plot(**plot_args)

## Merge the pdfs together

In [ ]:
# path where files are stored
pdfs_loc = path + 'detailedplots/'
# list all the files
flist = sorted(glob.glob(os.path.join(pdfs_loc, '*')))

In [ ]:
from PyPDF2 import PdfFileMerger, PdfFileReader
 
# Call the PdfFileMerger
mergedObject = PdfFileMerger()
 
# I had 116 files in the folder that had to be merged into a single document
# Loop through all of them and append their pages
for filename in flist:
    mergedObject.append(PdfFileReader(filename, 'rb'))
    
#  Write all the files into a file which is named as shown below
mergedObject.write(path+'DetailedPredvsTarget.pdf')

### Code cemetery

In [ ]:
# from keras.utils import plot_model
# plot_model(nn_model.model, to_file='model.png',show_shapes=True,)


# # Check the trainable status of the individual layers
# for layer in nn_model.model.layers:
#     print(layer, layer.trainable, layer.name)


#model.model.summary()


#nn_model.model.layers[-1].input_shape
#model.model.layers[-1].kernel.initializer.run(session=K.get_session())
#model.model.layers[-1].bias.initializer.run(session=K.get_session())
#nn_model.model.layers[-1].output_shape